In [4]:
import tensorflow as tf

graph = tf.Graph()

with graph.as_default():
    
    with tf.name_scope("variables"):
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name="global_step") # keep track of N run-times
        total_output = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="total_output") # sum of outputs
        
    with tf.name_scope("transformation"):
        with tf.name_scope("input"):
            a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a") # placeholder for a Vector
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a, name="product_b") # calculate product of all elements in the Vector a
            c = tf.reduce_sum(a, name="sum_c") # calculate sum of all elements in the Vector a
        with tf.name_scope("output"):
            output = tf.add(b, c, name="output")
    
    with tf.name_scope("update"):
        update_total = total_output.assign_add(output) # increase the total_output by the latest output
        increment_step = global_step.assign_add(1) # increase the global_step by 1
        
    with tf.name_scope("summaries"):
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average")
        tf.summary.scalar("Output", output)
        tf.summary.scalar("SumOfOutputsOverTime", update_total)
        tf.summary.scalar("AverageOfOutputsOverTime", avg)
        
    with tf.name_scope("global_ops"):
        init = tf.global_variables_initializer() # init all variables (in the above scope)
        merged_summaries = tf.summary.merge_all() # merge all summaries (in the above scope) into 1 operation
        
sess = tf.Session(graph=graph)
writer = tf.summary.FileWriter('./improved_graph', graph)

sess.run(init) # execute the correponding 'init' function


# ---
def run_graph(input_tensor):
    feed_dict = {a: input_tensor} # create a dictionary to map the input_tensor to the 'a'
    _, step, summary = sess.run([output, increment_step, merged_summaries], feed_dict=feed_dict) # execute the functions
                    # the return [_, step, summary] corresponds to the input [output, increment_step, merge_summaries]
    writer.add_summary(summary, global_step=step)
    
    #print(step)
    

# ---
run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])

# ---
writer.flush() # write the summaries to disk
writer.close()
sess.close()

1
2
3
4
5
6
7
8
9
10
